In [1]:
import codecs
import string
import sys
import numpy as np
import pandas as pd
import spacy
from spacy.lang.en import English
from spacy.tokens import Doc, Span, Token
from tqdm import tqdm

nlp = spacy.load('en')

In [13]:
from scipy.spatial.distance import cosine

doc = nlp('Harry wed Sally')
doc2 = nlp('Harry married Sally')
doc3 = nlp('Harry planned to marry Sally')
doc4 = nlp('Harry killed Sally')
doc5 = nlp('apple banana')

if not doc[1].has_vector:
    print('wed doesnt have a vector')
if not doc2[1].has_vector:
    print('married doesnt have a vector')
if not doc3[3].has_vector:
    print('marry doesnt have a vector')
if not doc4[1].has_vector:
    print('killed doesnt have a vector')
    
print(cosine(doc[1].vector, doc2[1].vector))
print(cosine(doc[1].vector, doc3[3].vector))
print(cosine(doc[1].vector, doc4[1].vector))

print(cosine(doc4[1].vector, doc3[3].vector))

print(cosine(doc5[0].vector, doc5[1].vector))

0.9974807724356651
1.1757262498140335
1.0077647855505347
0.8497251868247986
0.6314107775688171


In [5]:
docs_index = pd.read_csv('../data/documents.csv')
docs_index.head()

,document_id,set,kind,story_url,story_file_size,wiki_url,wiki_title,story_word_count,story_start,story_end
0,0025577043f5090cd603c6aea60f26e236195594,test,movie,http://www.awesomefilm.com/script/pumpupthevol...,54078,http://en.wikipedia.org/wiki/Pump_Up_the_Volum...,Pump Up the Volume (film),11499,Happy Harry Hardon,by Martin Eaves
1,0029bdbe75423337b551e42bb31f9a102785376f,train,gutenberg,http://www.gutenberg.org/ebooks/21572.txt.utf-8,814507,http://en.wikipedia.org/wiki/Percival_Keene,Percival Keene,173334,Produced by Nick,new eBooks .
2,00936497f5884881f1df23f4834f6739552cee8b,train,gutenberg,http://www.gutenberg.org/ebooks/3526.txt.utf-8,566874,http://en.wikipedia.org/wiki/Five_Weeks_in_a_B...,Five Weeks in a Balloon,112898,Produced by Judy,new eBooks .
3,00950a3641e6a28b04a6fabf6334140e2deaa9fd,train,gutenberg,http://www.gutenberg.org/ebooks/42188.txt.utf-8,90192,http://en.wikipedia.org/wiki/Shadows_in_the_Mo...,Shadows in the Moonlight (story),17670,Produced by Greg,new eBooks .
4,00ee9e01a0e581e0d8cbf7e865a895147c480c5e,train,movie,http://www.imsdb.com/scripts/Crank.html,309143,http://en.wikipedia.org/wiki/Crank_(film),Crank (film),27546,CRANK Written by,TO SOUNDTRACK .


In [6]:
for i in docs_index.head()['wiki_url']:
    print(i)

http://en.wikipedia.org/wiki/Pump_Up_the_Volume_(film)
http://en.wikipedia.org/wiki/Percival_Keene
http://en.wikipedia.org/wiki/Five_Weeks_in_a_Balloon
http://en.wikipedia.org/wiki/Shadows_in_the_Moonlight_(story)
http://en.wikipedia.org/wiki/Crank_(film)


In [13]:
questions = pd.read_csv('../data/qaps.csv')
questions.head(10)

,document_id,set,question,answer1,answer2,question_tokenized,answer1_tokenized,answer2_tokenized
0,0025577043f5090cd603c6aea60f26e236195594,test,Who is Mark Hunter?,He is a high school student in Phoenix.,A loner and outsider student with a radio stat...,Who is Mark Hunter ?,He is a high school student in Phoenix .,A loner and outsider student with a radio stat...
1,0025577043f5090cd603c6aea60f26e236195594,test,Where does this radio station take place?,It takes place in Mark's parents basement.,"Phoenix, Arizona",Where does this radio station take place ?,It takes place in Mark s parents basement .,"Phoenix , Arizona"
2,0025577043f5090cd603c6aea60f26e236195594,test,Why do more students tune into Mark's show?,Mark talks about what goes on at school and in...,Because he has a thing to say about what is ha...,Why do more students tune into Mark s show ?,Mark talks about what goes on at school and in...,Because he has a thing to say about what is ha...
3,0025577043f5090cd603c6aea60f26e236195594,test,Who commits suicide?,Malcolm.,Malcolm.,Who commits suicide ?,Malcolm .,Malcolm .
4,0025577043f5090cd603c6aea60f26e236195594,test,What does Paige jam into her microwave?,She jams her medals and accolades.,Her award medals,What does Paige jam into her microwave ?,She jams her medals and accolades .,Her award medals
5,0025577043f5090cd603c6aea60f26e236195594,test,What does Mark do with his radio station?,He dismantles it and attaches it to his mother...,Dismantle it.,What does Mark do with his radio station ?,He dismantles it and attaches it to his mother...,Dismantle it .
6,0025577043f5090cd603c6aea60f26e236195594,test,What does Mark tell the protesting students?,He tells them to make their own future.,That they should make their own future because...,What does Mark tell the protesting students ?,He tells them to make their own future .,That they should make their own future because...
7,0025577043f5090cd603c6aea60f26e236195594,test,Who gets arrested?,Mark and Nora.,Mark and Nora.,Who gets arrested ?,Mark and Nora .,Mark and Nora .
8,0025577043f5090cd603c6aea60f26e236195594,test,What does the radio show cause?,It causes trouble.,It causes much trouble in the community.,What does the radio show cause ?,It causes trouble .,It causes much trouble in the community .
9,0025577043f5090cd603c6aea60f26e236195594,test,Where does Mark Broadcast his station from?,Parent's Basement,At the basement of his home,Where does Mark Broadcast his station from ?,Parent s Basement,At the basement of his home


In [9]:
print(questions.loc[0,'question'])
print(questions.loc[0,'answer1'])
print(questions.loc[0,'answer2'])

Who is Mark Hunter?
He is a high school student in Phoenix.
A loner and outsider student with a radio station.


In [14]:
questions.loc[30:40]

,document_id,set,question,answer1,answer2,question_tokenized,answer1_tokenized,answer2_tokenized
30,0029bdbe75423337b551e42bb31f9a102785376f,train,Who does Arabella Mason wed?,"Ben Keene, Delmar's valet",Ben Keene,Who does Arabella Mason wed ?,"Ben Keene , Delmar s valet",Ben Keene
31,0029bdbe75423337b551e42bb31f9a102785376f,train,How does Percival Keene get his name?,"Percival is Captain Delmar's first name, and K...",Captain's name and Ben's surname,How does Percival Keene get his name ?,"Percival is Captain Delmar s first name , and ...",Captain s name and Ben s surname
32,0029bdbe75423337b551e42bb31f9a102785376f,train,Who is the bully that steals Percival's lunch?,"his teacher, Mr. O'Gallagher",The schoolmaster,Who is the bully that steals Percival s lunch ?,"his teacher , Mr. O'Gallagher",The schoolmaster
33,0029bdbe75423337b551e42bb31f9a102785376f,train,How does Percival get even with O'Gallagher af...,He sets them on fire with the teacher sitting ...,He sets them off underneath him,How does Percival get even with O'Gallagher af...,He sets them on fire with the teacher sitting ...,He sets them off underneath him
34,0029bdbe75423337b551e42bb31f9a102785376f,train,Who does Percival convince the Pirates to spare?,a rich Dutch merchant and his daughter Minnie,A Dutch Merchant and his daughter,Who does Percival convince the Pirates to spare ?,a rich Dutch merchant and his daughter Minnie,A Dutch Merchant and his daughter
35,0029bdbe75423337b551e42bb31f9a102785376f,train,How does Percival save Captain Delmar's life?,"When the captain is ill, Percival takes his pl...",Take his place in a duel,How does Percival save Captain Delmar s life ?,"When the captain is ill , Percival takes his p...",Take his place in a duel
36,0029bdbe75423337b551e42bb31f9a102785376f,train,What shocking news does Percival's mother admi...,Captain Delmar is Percival's father,He's a bastard,What shocking news does Percival s mother admi...,Captain Delmar is Percival s father,He s a bastard
37,0029bdbe75423337b551e42bb31f9a102785376f,train,What happens when Percival is captured by the ...,he is sentenced to execution,He escapes,What happens when Percival is captured by the ...,he is sentenced to execution,He escapes
38,0029bdbe75423337b551e42bb31f9a102785376f,train,What news does Percival receive at the end of ...,He has been granted the right to use his fathe...,He gets the arms and name of his biological fa...,What news does Percival receive at the end of ...,He has been granted the right to use his fathe...,He gets the arms and name of his biological fa...
39,0029bdbe75423337b551e42bb31f9a102785376f,train,What is the name of the de Versely family house?,Madeline Hall,Madeline Hall,What is the name of the de Versely family house ?,Madeline Hall,Madeline Hall


In [9]:
docs_index.loc[lambda df: df['document_id'] == '37c11f984cb14401d85abfc20e8305ca7a472c9f']

,document_id,set,kind,story_url,story_file_size,wiki_url,wiki_title,story_word_count,story_start,story_end
352,37c11f984cb14401d85abfc20e8305ca7a472c9f,train,movie,http://www.dailyscript.com/scripts/flightplan....,184942,http://en.wikipedia.org/wiki/Flightplan,Flightplan,28192,FLIGHTPLAN by Peter,-- THE END


In [11]:
docs_index.loc[lambda df: df['story_url'].str.contains('dailyscript')]

,document_id,set,kind,story_url,story_file_size,wiki_url,wiki_title,story_word_count,story_start,story_end
46,069ba0ceb78ef8fe8a193b7b4f225938f2044f0c,test,movie,http://www.dailyscript.com/scripts/demolition-...,238359,http://en.wikipedia.org/wiki/Demolition_Man_(f...,Demolition Man (film),32723,DEMOLITION MAN Participating,. THE END
260,291163d04045b7a50f6cfcb36f1541582d264208,train,movie,http://www.dailyscript.com/scripts/ford-fairla...,204946,http://en.wikipedia.org/wiki/The_Adventures_of...,The Adventures of Ford Fairlane,28029,FORD FAIRLANE Screenplay,did you ?
352,37c11f984cb14401d85abfc20e8305ca7a472c9f,train,movie,http://www.dailyscript.com/scripts/flightplan....,184942,http://en.wikipedia.org/wiki/Flightplan,Flightplan,28192,FLIGHTPLAN by Peter,-- THE END
412,405df1ccf0409ea6040c2a765a0315878f991d79,train,movie,http://www.dailyscript.com/scripts/goodfellas....,263699,http://en.wikipedia.org/wiki/Goodfellas,Goodfellas,30185,GOOD FELLAS by,: THE END
509,518bfcb6ea413715080cbd6f97aa479ac5f6c7d3,train,movie,http://www.dailyscript.com/scripts/last-of-the...,261529,http://en.wikipedia.org/wiki/The_Last_of_the_M...,The Last of the Mohicans (1992 film),30734,THE LAST OF,: THE END
566,5a284f4743e8864970dde010c6f0ce08fa787090,train,movie,http://www.dailyscript.com/scripts/beetlejuice...,222638,http://en.wikipedia.org/wiki/Beetlejuice,Beetlejuice,29174,BEETLE JUICE SECOND,FADE OUT .
832,8afdb338362e2cf594bb0cdf1f3aef8a982832ed,train,movie,http://www.dailyscript.com/scripts/hannibal_pr...,250085,http://en.wikipedia.org/wiki/Hannibal_(film),Hannibal (film),38444,H A N,FADE TO BLACK
847,8c9ff15b915f06f83304c45f9d84c102700ef8fb,train,movie,http://www.dailyscript.com/scripts/blair-witch...,148544,http://en.wikipedia.org/wiki/Book_of_Shadows:_...,Book of Shadows: Blair Witch 2,19130,BLAIR WITCH II,God -- END
871,91075ab5383e8113d86b17536c6918bfbbd6af20,train,movie,http://www.dailyscript.com/scripts/the-hauntin...,232971,http://en.wikipedia.org/wiki/The_Haunting_(199...,The Haunting (1999 film),36862,THE HAUNTING OF,FADE OUT .
1171,c0d43726bc642a4ece9affb11d27c5370544cb91,train,movie,http://www.dailyscript.com/scripts/batman-retu...,197407,http://en.wikipedia.org/wiki/Batman_Returns,Batman Returns,28185,BATMAN RETURNS by,. THE END


# Entity fix

In [3]:
text = 'The BBC producer allegedly struck by Jeremy Clarkson will not press charges against the “Top Gear” host, his lawyer said Friday. Clarkson, who hosted one of the most-watched television shows in the world, was dropped by the BBC Wednesday after an internal investigation by the British broadcaster found he had subjected producer Oisin Tymon “to an unprovoked physical and verbal attack.'

In [4]:
doc = nlp(text)

for ent in doc.ents:
    print(ent.text, ent.start_char, ent.end_char, ent.label_)

BBC 4 7 ORG
Jeremy Clarkson 37 52 PERSON
the “Top Gear” 84 98 LAW
Friday 121 127 DATE
Clarkson 129 137 ORG
one 150 153 CARDINAL
BBC 224 227 ORG
Wednesday 228 237 DATE
British 277 284 NORP
Oisin Tymon 329 340 PERSON


In [12]:
asdf = doc[1]
asdf.ent_id_, asdf.ent_iob_, asdf.ent_type_

('', 'B', 'ORG')

In [ ]:
def entity_doc_to_text(doc):
    # spacy issue #2073
    # if not doc[0].has_extension('entity_id'):
    #     print('Tokens don\'t have attribute: entity_id')
    #     return None
    
    text = ''
    for token in doc:
        if token._.entity_id is not None:
            whitespace = token.text_with_ws[len(token.text):]
            text += token._.entity_id + whitespace
        else:
            text += token.text_with_ws
    return text

In [79]:
import string
import numpy as np
from spacy.tokens import Doc, Span, Token

# simple heuristic
def simple_entity_detector(doc):
    upper = set()
    lower = set()

    for token in doc:
        if token.text[0] in string.ascii_uppercase:
            upper.add(token.text)
        else:
            lower.add(token.text)

    entities = [token for token in upper if token.lower() not in lower]
    entity_map = dict()
    Token.set_extension('entity_id')
    entity_ids = np.random.choice(10000, 10000).tolist()
    
    for i, token in enumerate(doc):
        if token.text in entities:
            if token.text not in entity_map.keys():
                if len(entity_ids) == 0:
                    print('Ran out of entity ids')
                entity_map[token.text] = '@entity' + str(entity_ids.pop())
            doc[i]._.entity_id = entity_map[token.text]
            
    return doc

new_doc = simple_entity_detector(doc)

In [93]:
def entity_doc_to_text(doc):
    # spacy issue #2073
    # if not doc[0].has_extension('entity_id'):
    #     print('Tokens don\'t have attribute: entity_id')
    #     return None
    
    text = ''
    for token in doc:
        if token._.entity_id is not None:
            whitespace = token.text_with_ws[len(token.text):]
            text += token._.entity_id + whitespace
        else:
            text += token.text_with_ws
    return text

new_text = entity_doc_to_text(new_doc)

In [94]:
doc

The BBC producer allegedly struck by Jeremy Clarkson will not press charges against the “Top Gear” host, his lawyer said Friday. Clarkson, who hosted one of the most-watched television shows in the world, was dropped by the BBC Wednesday after an internal investigation by the British broadcaster found he had subjected producer Oisin Tymon “to an unprovoked physical and verbal attack.

In [95]:
new_text

'The @entity835 producer allegedly struck by @entity5853 @entity53 will not press charges against the “@entity2756 @entity7677” host, his lawyer said @entity4068. @entity53, who hosted one of the most-watched television shows in the world, was dropped by the @entity835 @entity9835 after an internal investigation by the @entity1210 broadcaster found he had subjected producer @entity7026 @entity4680 “to an unprovoked physical and verbal attack.'

In [43]:
for token in doc:
    print(token.text, token.pos_, token.tag_, len(token.text), len(token.text_with_ws), len(token.text_with_ws[len(token.text):]))

The DET DT 3 4 1
BBC PROPN NNP 3 4 1
producer NOUN NN 8 9 1
allegedly ADV RB 9 10 1
struck VERB VBN 6 7 1
by ADP IN 2 3 1
Jeremy PROPN NNP 6 7 1
Clarkson PROPN NNP 8 9 1
will VERB MD 4 5 1
not ADV RB 3 4 1
press VERB VB 5 6 1
charges NOUN NNS 7 8 1
against ADP IN 7 8 1
the DET DT 3 4 1
“ PUNCT `` 1 1 0
Top PROPN NNP 3 4 1
Gear PROPN NNP 4 4 0
” PUNCT '' 1 2 1
host NOUN NN 4 4 0
, PUNCT , 1 2 1
his ADJ PRP$ 3 4 1
lawyer NOUN NN 6 7 1
said VERB VBD 4 5 1
Friday PROPN NNP 6 6 0
. PUNCT . 1 2 1
Clarkson PROPN NNP 8 8 0
, PUNCT , 1 2 1
who NOUN WP 3 4 1
hosted VERB VBD 6 7 1
one NUM CD 3 4 1
of ADP IN 2 3 1
the DET DT 3 4 1
most ADV RBS 4 4 0
- PUNCT HYPH 1 1 0
watched VERB VBN 7 8 1
television NOUN NN 10 11 1
shows NOUN NNS 5 6 1
in ADP IN 2 3 1
the DET DT 3 4 1
world NOUN NN 5 5 0
, PUNCT , 1 2 1
was VERB VBD 3 4 1
dropped VERB VBN 7 8 1
by ADP IN 2 3 1
the DET DT 3 4 1
BBC PROPN NNP 3 4 1
Wednesday PROPN NNP 9 10 1
after ADP IN 5 6 1
an DET DT 2 3 1
internal ADJ JJ 8 9 1
investigation NO